
### Running common notebook to get access to variables

In [0]:
%run "/Users/shanmukh@shanmukhsattiraju.com/04. Common"

In [0]:
dbutils.widgets.text(name="env",defaultValue='',label='Enter the environment in lower case')
env = dbutils.widgets.get("env")


### Reading from bronze raw_Roads

In [0]:

def read_BronzeRoadsTable(environment):
    print('Reading the Bronze Table raw_roads Data : ',end='')
    df_bronzeRoads = (spark.readStream
                    .table(f"`{environment}_catalog`.`bronze`.raw_roads")
                    )
    print(f'Reading {environment}_catalog.bronze.raw_roads Success!')
    print("**********************************")
    return df_bronzeRoads

In [0]:
df_roads = read_BronzeRoadsTable(env)

Reading the Bronze Table raw_roads Data : Reading dev_catalog.bronze.raw_roads Success!
**********************************



### Creating road_category_name column

In [0]:
def road_Category(df):
    print('Creating Road Category Name Column: ', end='')
    from pyspark.sql.functions import when,col

    df_road_Cat = df.withColumn("Road_Category_Name",
                  when(col('Road_Category') == 'TA', 'Class A Trunk Road')
                  .when(col('Road_Category') == 'TM', 'Class A Trunk Motor')
                   .when(col('Road_Category') == 'PA','Class A Principal road')
                    .when(col('Road_Category') == 'PM','Class A Principal Motorway')
                    .when(col('Road_Category') == 'M','Class B road')
                    .otherwise('NA')
                  
                  )
    print('Success!! ')
    print('***********************')
    return df_road_Cat


### Creating road_type column



In [0]:
def road_Type(df):
    print('Creating Road Type Name Column: ', end='')
    from pyspark.sql.functions import when,col

    df_road_Type = df.withColumn("Road_Type",
                  when(col('Road_Category_Name').like('%Class A%'),'Major')
                  .when(col('Road_Category_Name').like('%Class B%'),'Minor')
                    .otherwise('NA')
                  
                  )
    print('Success!! ')
    print('***********************')
    return df_road_Type


### Writing data to silver_roads in Silver schema

In [0]:
def write_Roads_SilverTable(StreamingDF,environment):
    print('Writing the silver_roads Data : ',end='') 

    write_StreamSilver_R = (StreamingDF.writeStream
                .format('delta')
                .option('checkpointLocation',checkpoint+ "/SilverRoadsLoad/Checkpt/")
                .outputMode('append')
                .queryName("SilverRoadsWriteStream")
                .trigger(availableNow=True)
                .toTable(f"`{environment}_catalog`.`silver`.`silver_roads`"))
    
    write_StreamSilver_R.awaitTermination()
    print(f'Writing `{environment}_catalog`.`silver`.`silver_roads` Success!')


### Calling functions

In [0]:
df_noDups = remove_Dups(df_roads)

AllColumns = df_noDups.schema.names
df_clean = handle_NULLs(df_noDups,AllColumns)

## Creating Road_Category_name 
df_roadCat = road_Category(df_clean)

## Creating Road_Type column
df_type = road_Type(df_roadCat)

## Writing data to silver_roads table

write_Roads_SilverTable(df_type,env)

Removing Duplicate values: Success!
Replacing NULLs of Strings DataType with "Unknown": Success!
Replacing NULLs of Numeric DataType with "0":  Success!
Creating Road Category Name Column: Success!! 
***********************
Creating Road Type Name Column: Success!! 
***********************
Writing the silver_roads Data : Writing `dev_catalog`.`silver`.`silver_roads` Success!


In [0]:
%sql

SELECT * FROM `dev_catalog`.`silver`.`silver_roads` 

Road_ID,Road_Category_Id,Road_Category,Region_ID,Region_Name,Total_Link_Length_Km,Total_Link_Length_Miles,All_Motor_Vehicles,Road_Category_Name,Road_Type
75,1,TM,7,East of England,245.9,152.8,4.120104199E9,Class A Trunk Motor,Major
44,5,M,10,West Midlands,27956.1,17371.12,9.927697665E9,Class B road,Minor
28,4,PA,7,East of England,2719.344,1689.72,7.493977513E9,Class A Principal road,Major
33,4,PA,8,Yorkshire and the Humber,2559.104,1590.15,6.365559059E9,Class A Principal road,Major
40,1,TM,10,West Midlands,351.963,218.7,5.280343918E9,Class A Trunk Motor,Major
39,5,M,9,South East,40851.1,25383.7,1.5597621429E10,Class B road,Minor
55,3,TA,2,East Midlands,1215.788,755.46,5.056721458E9,Class A Trunk Road,Major
52,4,PA,1,South West,3882.154,2412.26,8.028746829E9,Class A Principal road,Major
71,1,TM,6,London,66.386,41.25,1.311689053E9,Class A Trunk Motor,Major
66,1,TM,5,North West,558.839,347.25,7.370068617E9,Class A Trunk Motor,Major
